<a href="https://colab.research.google.com/github/vaishnavibonala10-gif/team-cartoon/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit pyngrok -q

In [11]:
from pyngrok import ngrok
ngrok.set_auth_token("367NCuvfIvmg0xRkJbcZ8WjaHSP_XRSRGHtbG4kQ2Begf1VG")

In [13]:
%%writefile fraud_app.py
import streamlit as st
import pandas as pd
from datetime import datetime
import hashlib
import time

# Page configuration
st.set_page_config(page_title="Fraud Detection System", page_icon="🛡️", layout="wide")

# Custom CSS
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        color: #667eea;
        text-align: center;
        margin-bottom: 2rem;
    }
    .stat-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 20px;
        border-radius: 10px;
        color: white;
        text-align: center;
    }
    .fraud-alert {
        background-color: #f8d7da;
        color: #721c24;
        padding: 15px;
        border-radius: 8px;
        border: 2px solid #f5c6cb;
        font-weight: bold;
        animation: blink 1s infinite;
    }
    .safe-alert {
        background-color: #d4edda;
        color: #155724;
        padding: 15px;
        border-radius: 8px;
        border: 2px solid #c3e6cb;
        font-weight: bold;
    }
    @keyframes blink {
        0%, 50%, 100% { opacity: 1; }
        25%, 75% { opacity: 0.5; }
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state
if 'users_db' not in st.session_state:
    st.session_state.users_db = {
        'demo': {
            'password': hashlib.sha256('demo123'.encode()).hexdigest(),
            'email': 'demo@example.com'
        }
    }

if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.username = None
    st.session_state.transactions = []
    st.session_state.page = 'login'

if 'currency' not in st.session_state:
    st.session_state.currency = 'USD'

# Fraud Detection Algorithm
def detect_fraud(amount, merchant, location):
    score = 0

    if amount > 1000:
        score += 0.3
    if amount > 5000:
        score += 0.4

    suspicious_merchants = ['unknown', 'offshore', 'crypto', 'foreign', 'anonymous', 'untrusted']
    if any(s in merchant.lower() for s in suspicious_merchants):
        score += 0.5

    suspicious_locations = ['unknown', 'overseas', 'foreign', 'offshore', 'anonymous']
    if any(s in location.lower() for s in suspicious_locations):
        score += 0.3

    import random
    score += random.random() * 0.2

    is_fraud = score > 0.6
    return is_fraud, min(score, 1.0)

# Currency conversion
def convert_currency(amount, from_curr, to_curr):
    if from_curr == to_curr:
        return amount
    elif from_curr == 'USD' and to_curr == 'INR':
        return amount * 83
    elif from_curr == 'INR' and to_curr == 'USD':
        return amount / 83
    return amount

def format_currency(amount, currency):
    if currency == 'USD':
        return f"${amount:,.2f}"
    else:
        return f"₹{amount:,.2f}"

# Login Page
def login_page():
    st.markdown("<h1 class='main-header'>🛡️ Fraud Detection System</h1>", unsafe_allow_html=True)

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.markdown("### 🔐 Login")

        username = st.text_input("Username", key="login_username")
        password = st.text_input("Password", type="password", key="login_password")

        col_btn1, col_btn2 = st.columns(2)

        with col_btn1:
            if st.button("Login", use_container_width=True):
                hashed_password = hashlib.sha256(password.encode()).hexdigest()

                if username in st.session_state.users_db:
                    if st.session_state.users_db[username]['password'] == hashed_password:
                        st.session_state.logged_in = True
                        st.session_state.username = username
                        st.session_state.page = 'dashboard'

                        if username == 'demo' and len(st.session_state.transactions) == 0:
                            st.session_state.transactions = [
                                {'id': 1, 'amount': 150.50, 'merchant': 'Amazon', 'location': 'New York',
                                 'timestamp': datetime.now().isoformat(), 'is_fraud': False, 'score': 0.15, 'currency': 'USD'},
                                {'id': 2, 'amount': 8500, 'merchant': 'Crypto Exchange', 'location': 'Unknown',
                                 'timestamp': datetime.now().isoformat(), 'is_fraud': True, 'score': 0.85, 'currency': 'USD'},
                                {'id': 3, 'amount': 3750, 'merchant': 'Flipkart', 'location': 'Mumbai',
                                 'timestamp': datetime.now().isoformat(), 'is_fraud': False, 'score': 0.05, 'currency': 'INR'},
                                {'id': 4, 'amount': 3200, 'merchant': 'Offshore Vendor', 'location': 'Foreign',
                                 'timestamp': datetime.now().isoformat(), 'is_fraud': True, 'score': 0.92, 'currency': 'USD'},
                            ]

                        st.success("✅ Login successful!")
                        time.sleep(1)
                        st.rerun()
                    else:
                        st.error("❌ Invalid credentials!")
                else:
                    st.error("❌ User not found!")

        with col_btn2:
            if st.button("Register", use_container_width=True):
                st.session_state.page = 'register'
                st.rerun()

        st.info("**📝 Demo Credentials:**\n\nUsername: `demo`\n\nPassword: `demo123`")

# Register Page
def register_page():
    st.markdown("<h1 class='main-header'>🛡️ Fraud Detection System</h1>", unsafe_allow_html=True)

    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.markdown("### 📝 Register New Account")

        username = st.text_input("Username", key="reg_username")
        email = st.text_input("Email", key="reg_email")
        password = st.text_input("Password", type="password", key="reg_password")
        confirm_password = st.text_input("Confirm Password", type="password", key="reg_confirm")

        col_btn1, col_btn2 = st.columns(2)

        with col_btn1:
            if st.button("Register", use_container_width=True):
                if not username or not email or not password:
                    st.error("❌ All fields are required!")
                elif password != confirm_password:
                    st.error("❌ Passwords do not match!")
                elif username in st.session_state.users_db:
                    st.error("❌ Username already exists!")
                else:
                    st.session_state.users_db[username] = {
                        'password': hashlib.sha256(password.encode()).hexdigest(),
                        'email': email
                    }
                    st.success("✅ Registration successful! Please login.")
                    time.sleep(2)
                    st.session_state.page = 'login'
                    st.rerun()

        with col_btn2:
            if st.button("Back to Login", use_container_width=True):
                st.session_state.page = 'login'
                st.rerun()

# Dashboard Page
def dashboard_page():
    col1, col2 = st.columns([3, 1])
    with col1:
        st.markdown(f"<h1 class='main-header'>🛡️ Fraud Detection Dashboard</h1>", unsafe_allow_html=True)
        st.markdown(f"**Welcome, {st.session_state.username}!** 👋")
    with col2:
        st.markdown("<br>", unsafe_allow_html=True)
        if st.button("🚪 Logout", use_container_width=True):
            st.session_state.logged_in = False
            st.session_state.username = None
            st.session_state.page = 'login'
            st.session_state.transactions = []
            st.rerun()

    st.session_state.currency = st.selectbox("💱 Select Currency", ['USD', 'INR'],
                                              index=0 if st.session_state.currency == 'USD' else 1)

    total_trans = len(st.session_state.transactions)
    fraud_count = sum(1 for t in st.session_state.transactions if t['is_fraud'])
    safe_count = total_trans - fraud_count

    total_amount = sum(
        convert_currency(t['amount'], t.get('currency', 'USD'), st.session_state.currency)
        for t in st.session_state.transactions
    )

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.markdown(f"""
        <div class='stat-card'>
            <h3>📊 Total</h3>
            <h1>{total_trans}</h1>
            <p>Transactions</p>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        st.markdown(f"""
        <div class='stat-card' style='background: linear-gradient(135deg, #dc3545 0%, #c82333 100%);'>
            <h3>⚠️ Fraud</h3>
            <h1>{fraud_count}</h1>
            <p>Detected</p>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        st.markdown(f"""
        <div class='stat-card' style='background: linear-gradient(135deg, #28a745 0%, #218838 100%);'>
            <h3>✓ Safe</h3>
            <h1>{safe_count}</h1>
            <p>Transactions</p>
        </div>
        """, unsafe_allow_html=True)

    with col4:
        st.markdown(f"""
        <div class='stat-card'>
            <h3>💰 Total</h3>
            <h1>{format_currency(total_amount, st.session_state.currency)}</h1>
            <p>Amount</p>
        </div>
        """, unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    with st.expander("➕ Add New Transaction", expanded=False):
        with st.form("new_transaction"):
            col1, col2, col3 = st.columns(3)

            with col1:
                trans_currency = st.selectbox("Currency", ['USD', 'INR'])
                amount = st.number_input(f"Amount ({trans_currency})", min_value=0.0, step=0.01)

            with col2:
                merchant = st.text_input("Merchant Name")

            with col3:
                location = st.text_input("Location")

            submitted = st.form_submit_button("🔍 Process Transaction", use_container_width=True)

            if submitted:
                if amount > 0 and merchant and location:
                    is_fraud, score = detect_fraud(amount, merchant, location)

                    new_transaction = {
                        'id': len(st.session_state.transactions) + 1,
                        'amount': amount,
                        'merchant': merchant,
                        'location': location,
                        'timestamp': datetime.now().isoformat(),
                        'is_fraud': is_fraud,
                        'score': score,
                        'currency': trans_currency
                    }

                    st.session_state.transactions.insert(0, new_transaction)

                    if is_fraud:
                        st.markdown(f"""
                        <div class='fraud-alert'>
                            🚨 FRAUD ALERT! 🚨<br>
                            This transaction has been flagged as suspicious!<br>
                            Fraud Score: {score*100:.1f}%<br>
                            Transaction BLOCKED for review!
                        </div>
                        """, unsafe_allow_html=True)
                        st.error(f"⚠️ Transaction BLOCKED: {merchant} - {format_currency(amount, trans_currency)}")
                    else:
                        st.markdown(f"""
                        <div class='safe-alert'>
                            ✓ Transaction Approved<br>
                            This transaction appears to be legitimate.
                        </div>
                        """, unsafe_allow_html=True)
                        st.success(f"✓ Transaction Processed: {merchant} - {format_currency(amount, trans_currency)}")

                    time.sleep(2)
                    st.rerun()
                else:
                    st.error("❌ Please fill in all fields!")

    st.markdown("### 📋 Recent Transactions")

    if st.session_state.transactions:
        df_data = []
        for t in st.session_state.transactions:
            trans_currency = t.get('currency', 'USD')
            df_data.append({
                'ID': t['id'],
                'Amount': format_currency(t['amount'], trans_currency),
                'Merchant': t['merchant'],
                'Location': t['location'],
                'Date': datetime.fromisoformat(t['timestamp']).strftime('%Y-%m-%d %H:%M'),
                'Status': '⚠️ FRAUD' if t['is_fraud'] else '✓ Safe',
                'Score': f"{t['score']*100:.1f}%"
            })

        df = pd.DataFrame(df_data)

        def highlight_fraud(row):
            if '⚠️ FRAUD' in row['Status']:
                return ['background-color: #f8d7da'] * len(row)
            else:
                return ['background-color: #d4edda'] * len(row)

        st.dataframe(
            df.style.apply(highlight_fraud, axis=1),
            use_container_width=True,
            hide_index=True
        )
    else:
        st.info("No transactions yet. Add your first transaction above!")

# Main App Logic
def main():
    if not st.session_state.logged_in:
        if st.session_state.page == 'login':
            login_page()
        elif st.session_state.page == 'register':
            register_page()
    else:
        dashboard_page()

if __name__ == "__main__":
    main()


Overwriting fraud_app.py


In [14]:
!streamlit run fraud_app.py &>/dev/null&

In [15]:
public_url = ngrok.connect(8501)
print("🌐 Your Fraud Detection App is now live!")
print(f"🔗 Access it here: {public_url}")

🌐 Your Fraud Detection App is now live!
🔗 Access it here: NgrokTunnel: "https://interbranch-rosenda-enviably.ngrok-free.dev" -> "http://localhost:8501"
